In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [ ]:
import numpy as np
import tensorflow as tf
from IPython.display import clear_output
from matplotlib import pyplot as plt

from preference_aggregation import *

from tqdm.notebook import tqdm
from copy import deepcopy

%matplotlib inline

In [ ]:
# generating 2d data
data = np.array(np.random.randn(100, 2), dtype=np.float32)

In [ ]:
# showing the data
plt.title("Data")
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.scatter(data[:, 0], data[:, 1])
plt.show()

In [ ]:
# expert configuration
n_experts = 2
models = [PreferenceLearningModel() for _ in range(n_experts)]

In [ ]:
def preference_fcn_1(idx_1, idx_2):
    """Preference for expert 1."""
    data_1 = data[idx_1, :]
    data_2 = data[idx_2, :]
    return np.array([data_1[0] > data_2[0]])


def preference_fcn_2(idx_1, idx_2):
    """Preference for expert 2."""
    data_1 = data[idx_1, :]
    data_2 = data[idx_2, :]
    return np.array([data_1[1] > data_2[1]])


preference_fcns = [preference_fcn_1, preference_fcn_2]

In [ ]:
# fitting two models with respective preferences
for i in range(n_experts):
    for _ in tqdm(range(100)):
        models[i].iteration(data=data, override_input_fcn=preference_fcns[i], do_plot=False, do_fit=True)

In [ ]:
for i in range(n_experts):
    models[i].plot_loss_and_decisions(data)

In [ ]:
# aggregating data from two experts
aggregator = MedianPreferenceAggregator(models)
aggregator.plot_with_colors(data)

Kind of does what we want. Nevertheless, trying the better model with regularization

# Aggregated preference model with regularization

In [ ]:
# creating data
models = [PreferenceLearningModel(name='preference_%d' % i) for i in range(n_experts)]

# aggregating them
aggregator = MedianPreferenceAverageRegularizationAggregator(models)

In [ ]:
# populating models with data
for i in range(n_experts):
    for _ in tqdm(range(100)):
        models[i].iteration(override_input_fcn=preference_fcns[i],
                            data=data, do_plot=False, do_fit=False)

In [ ]:
# fitting the models
aggregator.fit(epochs=5000, reg_common_to_0=1e-5, reg_m_to_common=1e-5)

In [ ]:
# saving data
aggregator.save('.')
losses = deepcopy(aggregator.losses)

In [ ]:
# showing the results
aggregator.plot_loss_and_decisions(data)

In [ ]:
# creating data
models = [PreferenceLearningModel(name='preference_%d' % i) for i in range(n_experts)]

# aggregating them
aggregator = MedianPreferenceAverageRegularizationAggregator(models)
aggregator.losses = losses
aggregator.load('.')
aggregator.plot_loss_and_decisions(data)

#### Again, the result is correct (upper right corner)!